In [11]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

### 1. HuggingFace에 업로드된 임베딩 모델 (bge-m3)을 사용해보자.
- bge-m3는 BAAI (베이징 인공지능 연구소)에서 개발한 다기능 임베딩 모델로, 한국어를 포함한 다국어의 포괄적 지원이 가능한 특징이 있다.
- 2024년 2월 기준, 한국어 벤치마크에서 SOTA 성능을 달성한 바 있다.
- Reference: https://arxiv.org/pdf/2402.03216 (ACL 2024)

In [ ]:
model = SentenceTransformer('BAAI/bge-m3')

In [ ]:
model.encode('종현이는 수연이를 사모한다.')

In [ ]:
model.encode('종현이는 욱과 구 사이에서 마치 등 터진 새우와 같았다.')

In [8]:
def get_embedding(sentence):
    return list(model.encode(sentence))

In [ ]:
data = ['저는 배가 고파요',
        '저기 배가 지나가네요',
        '굶어서 허기가 지네요',
        '허기 워기라는 게임이 있는데 즐거워',
        '스팀에서 재밌는 거 해야지',
        '스팀에어프라이어로 연어구이 해먹을거야',
        '종현은 수연을 좋아한다']
df = pd.DataFrame(data, columns=['text'])
df

In [13]:
df['embedding'] = df['text'].apply(get_embedding)

In [ ]:
df

In [15]:
def cos_sim(a, b):
    return np.dot(a, b) / np.linalg.norm(a) * np.linalg.norm(b)

In [17]:
def return_answer_candidate(df, query, top_k=3):
    query_embedding = get_embedding(query)
    df['similarity'] = df['embedding'].apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))
    results = df.sort_values(by=['similarity'], ascending=False, ignore_index=True)[:top_k]
    return results

In [ ]:
sim_result = return_answer_candidate(df, '아무 것도 안 먹었더니 꼬르륵 소리가 나네')
sim_result

### 2. OpenAI가 유료로 제공하는 API 형태의 임베딩 모델 (text-embedding-ada-002)을 사용해보자.
- text-embedding-ada-002: 2022년 12월 출시된 비교적 저렴한 API 모델 (dim: 1536)
- text-similarity-babbage-001 (dim: 2048)
- ...

In [33]:
from openai import OpenAI
import openai

OPENAI_API_KEY = '당신의 키 값을 입력하세요. MLV 계정 키 값 발급/사용 금지!!!'

openai.api_key = OPENAI_API_KEY

In [35]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [31]:
def get_embedding_by_openai(sentence, model='text-embedding-ada-002'):
    res = client.embeddings.create(
        input=sentence,
        model=model,
    )
    return res.data[0].embedding

In [ ]:
get_embedding_by_openai('종현이는 수연이를 사모한다.')

In [ ]:
len(get_embedding_by_openai('종현이는 수연이를 사모한다.'))

In [ ]:
df

In [39]:
df['embedding'] = df['text'].apply(get_embedding_by_openai)

In [40]:
def return_answer_candidate_by_openai(df, query, top_k=3):
    query_embedding = get_embedding_by_openai(query)
    df['similarity'] = df['embedding'].apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))
    results = df.sort_values(by=['similarity'], ascending=False, ignore_index=True)[:top_k]
    return results

In [ ]:
sim_result = return_answer_candidate_by_openai(df, '아무 것도 안 먹었더니 꼬르륵 소리가 나네')
sim_result

In [ ]:
sim_result = return_answer_candidate_by_openai(df, '종현은 성헌을 그윽하게 본다.')
sim_result